In [1]:
import cloudknot as ck
import AFQ.data as afqd

/Users/richford/miniconda3/envs/cloudknot-pyafq/lib/python3.8/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
study = afqd.S3BIDSStudy(
    study_id="hbn-qsiprep",
    bucket="fcp-indi",
    s3_prefix="data/Projects/HBN/BIDS_curated/derivatives/qsiprep",
    anon=True,
    subjects=[1],
)

Retrieving subject S3 keys
[########################################] | 100% Completed |  0.5s


In [3]:
subjects = study._all_subjects
print(len(subjects))

2136


In [13]:
def create_niftis(subject):
    import AFQ.data as afqd
    import bids
    import numpy as np
    import os
    import os.path as op

    from dipy.io.image import load_nifti, save_nifti
    from dipy.io import read_bvals_bvecs
    from dipy.core.gradients import gradient_table
    import dipy.reconst.dti as dti
    from dipy.align import resample
    
    from s3fs import S3FileSystem
    
    study = afqd.S3BIDSStudy(
        study_id="hbn-qsiprep",
        bucket="fcp-indi",
        s3_prefix="data/Projects/HBN/BIDS_curated/derivatives/qsiprep",
        anon=True,
        subjects=[subject]
    )
    
    local_bids_folder = "hbn"
    output_bucket = "hbn-pod2-deep-learning"
    study.download(local_bids_folder)
    layout = bids.BIDSLayout(local_bids_folder, validate=False)
    
    subject = subject.replace("sub-", "")        

    bids_filters = {"subject": subject, "return_type": "filename"}

    fdwi = layout.get(extension="nii.gz", suffix="dwi", **bids_filters)[0]
    fb0 = layout.get(suffix="dwiref", extension="nii.gz", **bids_filters)[0]
    ft1w = layout.get(suffix="T1w", extension="nii.gz", space=None, **bids_filters)[0]
    fmask = layout.get(suffix="mask", datatype="dwi", extension="nii.gz", **bids_filters)[0]
    fwm = layout.get(suffix="probseg", space=None, extension="nii.gz", **bids_filters)
    fwm = [fn for fn in fwm if "label-WM" in fn][0]

    t1w_data, t1w_affine = load_nifti(ft1w)
    mask_data, mask_affine = load_nifti(fmask)
    b0_data, b0_affine = load_nifti(fb0)

    data, affine = load_nifti(fdwi)
    wm_data, wm_affine = load_nifti(fwm)
    t1w_dwi = resample(t1w_data, data[:, :, :, 0], t1w_affine, affine)
    wm_dwi = resample(wm_data, data[:, :, :, 0], wm_affine, affine)
    mask_dwi = resample(mask_data, data[:, :, :, 0], mask_affine, affine)
    b0_dwi = resample(b0_data, data[:, :, :, 0], b0_affine, affine)

    fbval = layout.get_bval(path=fdwi, subject=subject)
    fbvec = layout.get_bvec(path=fdwi, subject=subject)
    bvals, bvecs = read_bvals_bvecs(fbval, fbvec)
    gtab = gradient_table(bvals, bvecs)

    tenmodel = dti.TensorModel(gtab)
    tenfit = tenmodel.fit(data)
    FA = dti.fractional_anisotropy(tenfit.evals)
    FA = np.clip(FA, 0, 1)

    FA_masked = FA * wm_dwi.get_fdata()
    RGB = dti.color_fa(FA_masked, tenfit.evecs)

    RGB = np.array(255 * RGB, 'uint8')

    def trim_zeros(arr, margin=0, trim_dims=None):
        '''
        Trim the leading and trailing zeros from a N-D array.

        :param arr: numpy array
        :param margin: how many zeros to leave as a margin
        :returns: trimmed array
        :returns: slice object
        '''
        s = []
        if trim_dims is None:
            trim_dims = list(range(arr.ndim))

        for dim in range(arr.ndim):
            start = 0
            end = -1

            if dim in trim_dims:
                slice_ = [slice(None)]*arr.ndim

                go = True
                while go:
                    slice_[dim] = start
                    go = not np.any(arr[tuple(slice_)])
                    start += 1
                start = max(start-1-margin, 0)

                go = True
                while go:
                    slice_[dim] = end
                    go = not np.any(arr[tuple(slice_)])
                    end -= 1
                end = arr.shape[dim] + min(-1, end+1+margin) + 1

                s.append(slice(start,end))
            else:
                s.append(slice(None, None, None))
        return arr[tuple(s)], tuple(s)

    def pad_volume_to_128(arr, dim_max=128):
        dim_x, dim_y, dim_z = arr.shape[0], arr.shape[1], arr.shape[2]
        pad_xr = (dim_max - dim_x) // 2
        pad_xl = dim_max - dim_x - pad_xr
        pad_yr = (dim_max - dim_y) // 2
        pad_yl = dim_max - dim_y - pad_yr
        pad_zr = (dim_max - dim_z) // 2
        pad_zl = dim_max - dim_z - pad_zr

        pad_width = [(pad_xl, pad_xr), (pad_yl, pad_yr), (pad_zl, pad_zr)]
        for i in range(arr.ndim - 3):
            pad_width.append((0, 0))
        
        return np.pad(arr, pad_width=pad_width)

    mask_trim, trim_slices = trim_zeros(mask_dwi.get_fdata(), margin=5, trim_dims=(0, 1))
    t1w_dwi = t1w_dwi.get_fdata()[trim_slices]
    RGB = RGB[trim_slices + (slice(None, None, None),)]
    FA_masked = FA_masked[trim_slices]
    b0_dwi = b0_dwi.get_fdata()[trim_slices]

    is_128 = True
    
    try:
        RGB = pad_volume_to_128(RGB)
        FA_masked = pad_volume_to_128(FA_masked.astype(np.float32))
        b0_dwi = pad_volume_to_128(b0_dwi)
        is_128 = True
    except ValueError:
        dim_max = np.max([
            np.max(b0_dwi.shape),
            np.max(FA_masked.shape),
            np.max(RGB.shape), 
        ])
        RGB = pad_volume_to_128(RGB, dim_max=dim_max)
        FA_masked = pad_volume_to_128(FA_masked.astype(np.float32), dim_max=dim_max)
        b0_dwi = pad_volume_to_128(b0_dwi, dim_max=dim_max)
        is_128 = False

    combined = np.concatenate([
        RGB / 255,
        b0_dwi[..., np.newaxis] / np.max(b0_dwi)
    ], axis=-1)

    # Save and upload to S3
    fname_combined = f"sub-{subject}_combined.nii.gz"
    if is_128:
        save_nifti(fname_combined, combined, affine)
        fs = S3FileSystem(anon=False)
        fs.put(fname_combined, "/".join([output_bucket, "combined", op.basename(fname_combined)]))
    else:
        fname_combined = f"sub-{subject}_desc-irregularsize_b0colorfa.nii.gz"
        save_nifti(fname_combined, combined, affine)
        fs = S3FileSystem(anon=False)
        fs.put(fname_combined, "/".join([output_bucket, "combined", op.basename(fname_combined)]))


In [14]:
di = ck.DockerImage(
    func=create_niftis,
    base_image="python:3.8",
    github_installs="https://github.com/yeatmanlab/pyAFQ.git@master",
    overwrite=True
)

In [15]:
di.build(tags=["hbn-pod2-niftis-20210720"])

In [16]:
repo = ck.aws.DockerRepo(name=ck.get_ecr_repo())

In [17]:
# The very first time you run this, this command could take
# a few hours because the docker image is large
di.push(repo=repo)

In [9]:
# Specify bid_percentage to use Spot instances
# And make sure the volume size is large enough. 55-60 GB seems about right for HBN preprocessing. YMMV.
knot = ck.Knot(
    name=f"hbn-pod2-niftis-20210720-1",
    docker_image=di,
    pars_policies=("AmazonS3FullAccess",),
    bid_percentage=100,
    memory=8000,
    job_def_vcpus=4,
    max_vcpus=4096,
    retries=3,
    aws_resource_tags={"Project": "HBN-FCP-INDI"},
)

# Retrieve the above knot from config file
# knot = ck.Knot(name=f"fibr-gifs-20201116-0")

In [10]:
pilot_subs = subjects[:10]
production_subs = subjects[10:]

In [18]:
results = knot.map(pilot_subs)

In [21]:
knot.view_jobs()

Job ID              Name                        Status   
---------------------------------------------------------
662a71ee-0aca-42a4-93ac-8624cfed7d24        hbn-pod2-niftis-20210720-1-0        FAILED   
ef36175d-1684-4276-b588-93f8a9ac19e4        hbn-pod2-niftis-20210720-1-1        SUCCEEDED
ef850e26-6eed-4a15-bbb6-cecd648822be        hbn-pod2-niftis-20210720-1-2        SUCCEEDED


In [20]:
results = knot.map(production_subs)

In [22]:
knot.clobber(clobber_pars=True)